<a href="https://colab.research.google.com/github/JamesHemmings/TensorFLow_Binary_Classification_model/blob/main/realtime_face_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

import cv2

In [7]:
#open cv object to detect faces

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_default.xml")

In [8]:
#loading trained model


james_model = keras.models.load_model("models/saved_models/james_model.h5",
                                      custom_objects={'KerasLayer':hub.KerasLayer})  #had to add this because error was appearing 
class_names = ["me","not_me"]

In [9]:
#function copied from https://gitlab.com/Winston-90/me_not_me_detector/-/blob/main/me_not_me_detector.ipynb
#modified to create tensor in shape required for my model

def get_extended_image(img, x, y, w, h, k=0.1):
    '''
    Function, that return cropped image from 'img' that out trained model is able to use
    If k=0 returns image, cropped from (x, y) (top left) to (x+w, y+h) (bottom right)
    If k!=0 returns image, cropped from (x-k*w, y-k*h) to (x+k*w, y+(1+k)*h)
    After getting the desired image resize it to 250x250.
    And converts to tensor with shape (1, 250, 250, 3)

    Parameters:
        img (array-like, 2D): The original image
        x (int): x coordinate of the upper-left corner
        y (int): y coordinate of the upper-left corner
        w (int): Width of the desired image
        h (int): Height of the desired image
        k (float): The coefficient of expansion of the image

    Returns:
        image (tensor with shape (1, 224, 224, 3))
    '''

    # The next code block checks that coordinates will be non-negative
    # (in case if desired image is located in top left corner)
    if x - k*w > 0:
        start_x = int(x - k*w)
    else:
        start_x = x
    if y - k*h > 0:
        start_y = int(y - k*h)
    else:
        start_y = y

    end_x = int(x + (1 + k)*w)
    end_y = int(y + (1 + k)*h)

    face_image = img[start_y:end_y,
                     start_x:end_x]
    face_image = tf.image.resize(face_image, [224, 224])
    # shape from (250, 250, 3) to (1, 250, 250, 3)
    face_image = np.expand_dims(face_image, axis=0)
    return face_image


In [11]:
video = cv2.VideoCapture(0)
try:
  while True:
    
      # Capture the video frame
      # by frame
    ret, frame = video.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.3,
        minNeighbors = 5,
        minSize=(100,100),
        flags=cv2.CASCADE_SCALE_IMAGE
      )

    for (x,y,w,h) in faces:
      #for faces detected by opencv get the scaled image of this face that fits input shape of our model
      face_img = get_extended_image(img=frame,x=x,y=y,w=w,h=h,k=0.5)

      #predict on face then draw rectangle around the face
      prediction = james_model.predict(face_img)
      pred_label = class_names[round(prediction[0][0])]
      confidence = np.array(prediction[0]).max(axis=0)

      #color to make rectangle based on prediction
      if pred_label == "me":
        color = (0,255,0) 
      else:
        color = (255,0,0) 


      #draw the rectangle
      cv2.rectangle(frame,
                    (x,y),  #start point of rectangle
                    (x+w,y+h), #end point of rectangle
                    color,
                    2         #px thickness of rectangle
                    )

      cv2.putText(frame,
  f"{pred_label}, {round(100-prediction[0][0]*100,2)}%",
                  (x, y),
                  cv2.FONT_HERSHEY_PLAIN,  # font
                  2,  # fontScale
                  color,
                  2)  # thickness in px
                
    cv2.imshow("Face Detector",frame)
    
    # Exit with ESC
    key = cv2.waitKey(1)
    if key % 256 == 27:  # ESC code
      break

except Exception as e:
  print("failed")
  print(e)
  video.release()
  cv2.destroyAllWindows()

video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step


In [10]:
+